In [23]:
import pandas as pd
import numpy as np


np.random.seed(42)

## import data set
data = pd.read_csv('train.csv')
data['row_number'] = range(0,data.shape[0])

## Data shuffle
data = data.sample(frac=1,random_state=42)
tmp = pd.DataFrame()

## 600 entries per class
for label in range(10):
    if label==0:
        tmp = data[data['label']==label].head(600)
    else:
        curr = data[data['label']==label].head(600)
        tmp = pd.concat([tmp,curr])
data_train = tmp
trainRowNums = tmp['row_number'].values
test_set = data.loc[~data['row_number'].isin(trainRowNums)]

## Create one hot encoding
one_hot = pd.get_dummies(data_train['label'].unique())
one_hot['label'] = one_hot.index

data_train = pd.merge(data_train,one_hot)
data_test = test_set.sample(frac=1)
tmp = pd.DataFrame()

## Test set with 120 entries per class
for label in range(10):
    if label==0:
        tmp = data_test[data_test['label']==label].head(120)
    else:
        curr = data_test[data_test['label']==label].head(120)
        tmp = pd.concat([tmp,curr])
data_test = tmp
data_test = pd.merge(data_test,one_hot)
data_train.drop('label',axis=1,inplace=True)

data_test.drop('label',axis=1,inplace=True)

## Create the train and test set and normalize the inputs
# X_train = np.array(data_train.drop([0,1,2,3,4,5,6,7,8,9,'row_number'],axis=1).values)/255
# y_train = np.array(data_train[[0,1,2,3,4,5,6,7,8,9]].values)
# X_test = np.array(data_test.drop([0,1,2,3,4,5,6,7,8,9,'row_number'],axis=1).values)/255
# y_test = np.array(data_test[[0,1,2,3,4,5,6,7,8,9]].values)

In [24]:
# Normalize, and pixel entries range from 0 to 255
# First 10 cols are targets (ytrain/ytest) and the rest is input data
X_train = np.transpose(data_train.iloc[:, 11:].values / 255)
y_train = np.transpose(data_train.iloc[:, :10].values)
X_test = np.transpose(data_test.iloc[:, 11:].values / 255)
y_test = np.transpose(data_test.iloc[:, :10].values)
X_train_reshape = np.zeros((X_train.shape[1],1,28,28))
for i in range(X_train.shape[1]):
    temp = X_train[:,i]
    temp = np.ravel(temp)
    temp = temp.reshape(28,28)
    X_train_reshape[i,0,:,:] = temp
    
X_train= X_train_reshape  

X_test_reshape = np.zeros((X_test.shape[1],1,28,28))

In [25]:
# Re-shape the input vectors to 28X28 NumPy arrays
from matplotlib import pyplot as plt


for i in range(X_test.shape[1]):
    temp = X_test[:,i]
    temp = np.ravel(temp)
    temp = temp.reshape(28,28)
    X_test_reshape[i,0,:,:] = temp

X_test= X_test_reshape

conv1 = np.random.randn(2,1,5,5) * np.sqrt(1. / 5)

stride = 1
filter_h = conv1.shape[2]
filter_w = conv1.shape[3]
new_channels = conv1.shape[0]
## Get resultant Width and Height of Image

image_data = X_train
result_h = int(((image_data.shape[2]-filter_h)/stride) + 1)
result_w = int(((image_data.shape[3]-filter_w)/stride) + 1)

## Out 0 matrix
result_conv1 = np.random.rand(image_data.shape[0],new_channels,result_h,result_w)

# Iterate over each image in the dataset
for image_position in range(image_data.shape[0]):
    image_selected = image_data[image_position,:,:,:]
    
    # Each filter in conv layer
    for filter_position in range(conv1.shape[0]):
        filter_selected = conv1[filter_position,:,:,:]
        
        # Vertical slide across the image
        for i in range(0, image_selected.shape[1], stride):
            # Vertical slice matching filter height
            image_rectangle = image_selected[:,i:i+filter_h,:]
            # If its height is less than the filter's height skip
            if image_rectangle.shape[1] < filter_h:
                continue
            
            # Horizontal slide across the vertical slice
            for j in range(0, image_rectangle.shape[2], stride):
                # Slice matching the filter's width
                image_portion = image_rectangle[:,:,j:j+filter_w]
                # Skip if width less than the filter's width
                if image_portion.shape[2] < filter_w:
                    continue
                
                # Apply filter to selected image portion
                convolution_result = np.multiply(filter_selected, image_portion)
                # Sum convolution output and store it in the result array
                result_conv1[image_position, filter_position, i, j] = np.sum(convolution_result)


In [26]:
def im2col(X, conv1, stride, pad):
    # Pad around images to preserve size after convolution
    X_padded = np.pad(X, ((0, 0), (0, 0), (pad, pad), (pad, pad)), mode='constant')
    X = X_padded
    
    # Dims of the output post-filter
    new_height = int((X.shape[2] - conv1.shape[2]) / stride) + 1
    new_width = int((X.shape[3] - conv1.shape[3]) / stride) + 1
    
    # Arr for unique transformed matrix
    im2col_vector = np.zeros((X.shape[1] * conv1.shape[2] * conv1.shape[3], new_width * new_height * X.shape[0]))
    c = 0  # Curr column in the output array
    
    # Iterate over each image in the batch
    for position in range(X.shape[0]):
        # Current image
        image_position = X[position, :, :, :]
        
        # Vertical slide down the image, with "stride" steps 
        for height in range(0, image_position.shape[1], stride):
            # Select a horizontal slice that matches filter height
            image_rectangle = image_position[:, height:height + conv1.shape[2], :]
            # Next iteration if slice's height < than filter height
            if image_rectangle.shape[1] < conv1.shape[2]:
                continue
            
            # Horizontal slide down the image, with "stride" steps 
            for width in range(0, image_rectangle.shape[2], stride):
                # Slice portion that matches the filter width
                image_square = image_rectangle[:, :, width:width + conv1.shape[3]]
                # If portion's width < filter width
                if image_square.shape[2] < conv1.shape[3]:
                    continue
                
                # Flatten and store 
                im2col_vector[:, c:c + 1] = image_square.reshape(-1, 1)
                c += 1
 
    return im2col_vector


In [32]:
X = np.array([[1,0,0],[1,2,3],[3,4,5]])
X = X.reshape(1,1,3,3)
conv1 = np.array([[1,0],[0,1]])
conv1 = conv1.reshape(1,1,2,2)
X_im2col = im2col(X=X,conv1=conv1,stride=1,pad=0)
conv1_col = conv1.reshape(conv1.shape[0],-1)
X_result = conv1_col@X_im2col
X_result = np.array(np.hsplit(X_result, X.shape[0])).reshape((X.shape[0],conv1.shape[0],result_h,result_w))
print(X_result)

ValueError: cannot reshape array of size 4 into shape (1,1,24,24)

In [ ]:
def ReLU(x):
    return (x > 0) * x

def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x)

def maxpool_multiple(input_image, stride=2):
    # This function applies max pooling to a batch of images with a specified stride.
    # Calculate the dimensions of the output image
    input_width = input_image.shape[3]
    input_height = input_image.shape[2]
    filter_width = 2
    filter_height = 2
    
    output_width = int((input_width - filter_width) / stride) + 1
    output_height = int((input_height - filter_height) / stride) + 1
    
    # Initialize the output image array with zeros
    output_image = np.zeros((input_image.shape[0], input_image.shape[1], output_height, output_width))
    # Apply max pooling to each image in the batch
    for i in range(output_image.shape[0]):
        output_image[i:i+1, :, :, :] = maxpool(input_image[i:i+1, :, :, :], stride=2)
    return output_image

def maxpool(input_image, stride=2):
    # Apply max pooling to a single image with a specified stride and 2x2 pooling filter.
    input_width = input_image.shape[3]
    input_height = input_image.shape[2]
    filter_width = 2
    filter_height = 2
    n_channels = input_image.shape[1]
    num_images = input_image.shape[0]
    
    # Output dims
    output_width = int((input_width - filter_width) / stride) + 1
    output_height = int((input_height - filter_height) / stride) + 1
    
    # Initialize the output array with zeros
    output = np.zeros((n_channels, output_width * output_height))
    c = 0      
    # Iteration over input image height
    for height in range(0, input_height, stride):
        if height + filter_height <= input_height:
            # Vertical slice of the image
            image_rectangle = input_image[0, :, height:height + filter_height, :]
            # Width of the input image
            for width in range(0, input_width, stride):
                if width + filter_width <= input_width:
                    # Square portion of the image
                    image_square = image_rectangle[:, :, width:width + filter_width]
                    # Flattened square portion and max pool
                    image_flatten = image_square.reshape(-1, 1)
                    output[:, c:c + 1] = np.array([float(max(i)) for i in np.split(image_flatten, n_channels)]).reshape(-1, 1)
                    c += 1
   
    # Reshape output to match the expected
    final_output = np.array(np.hsplit(output, 1)).reshape((1, n_channels, output_height, output_width))
        
    return final_output


In [ ]:
def dReLU(x):
    return (x>0)*1.0
